<a href="https://colab.research.google.com/github/anquabkhan/google-collab/blob/main/Space_Missions_Analysis_(start).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Introduction

<center><img src="https://i.imgur.com/9hLRsjZ.jpg" height=400></center>

This dataset was scraped from [nextspaceflight.com](https://nextspaceflight.com/launches/past/?page=1) and includes all the space missions since the beginning of Space Race between the USA and the Soviet Union in 1957!

### Install Package with Country Codes

In [ ]:
%pip install iso3166

### Upgrade Plotly

Run the cell below if you are working with Google Colab.

In [ ]:
%pip install --upgrade plotly

### Import Statements

In [ ]:
import numpy as np
import pandas as pd
import plotly.express as px
import matplotlib.pyplot as plt
import seaborn as sns

# These might be helpful:

from iso3166 import countries
from datetime import datetime, timedelta

### Notebook Presentation

In [ ]:
pd.options.display.float_format = '{:,.2f}'.format

### Load the Data

In [ ]:
df_data = pd.read_csv('mission_launches.csv')

FileNotFoundError: ignored

# Preliminary Data Exploration

* What is the shape of `df_data`? 
* How many rows and columns does it have?
* What are the column names?
* Are there any NaN values or duplicates?

In [ ]:
print(df_data.shape)
print(df_data.columns)
print(df_data.isna().values.any())
print(df_data.duplicated().values.any())




In [ ]:
df_data

## Data Cleaning - Check for Missing Values and Duplicates

Consider removing columns containing junk data. 

In [ ]:
df_data2 = df_data.dropna(axis='columns')

## Descriptive Statistics

In [ ]:
df_data2.describe()

In [ ]:
print(df_data2.describe())
df_data2.info()

# Number of Launches per Company

Create a chart that shows the number of space mission launches by organisation.

In [ ]:
org_count = df_data2.value_counts('Organisation')
org_count

In [ ]:
fig = px.bar(data_frame=org_count, x=org_count.index, y=org_count.values, labels={ "org_count.index": "Number of launches"},height=500 )
fig.update_layout(yaxis_title = "Number of launches")



# Number of Active versus Retired Rockets

How many rockets are active compared to those that are decomissioned? 

In [ ]:
df_data2

In [ ]:
df_data2.value_counts('Rocket_Status')

# Distribution of Mission Status

How many missions were successful?
How many missions failed?

In [ ]:
df_data2.value_counts('Mission_Status')

# How Expensive are the Launches? 

Create a histogram and visualise the distribution. The price column is given in USD millions (careful of missing values). 

In [ ]:
df_data

In [ ]:
px.histogram(data_frame=df_data, x=df_data['Organisation'])

# Use a Choropleth Map to Show the Number of Launches by Country

* Create a choropleth map using [the plotly documentation](https://plotly.com/python/choropleth-maps/)
* Experiment with [plotly's available colours](https://plotly.com/python/builtin-colorscales/). I quite like the sequential colour `matter` on this map. 
* You'll need to extract a `country` feature as well as change the country names that no longer exist.

Wrangle the Country Names

You'll need to use a 3 letter country code for each country. You might have to change some country names.

* Russia is the Russian Federation
* New Mexico should be USA
* Yellow Sea refers to China
* Shahrud Missile Test Site should be Iran
* Pacific Missile Range Facility should be USA
* Barents Sea should be Russian Federation
* Gran Canaria should be USA


You can use the iso3166 package to convert the country names to Alpha3 format.

In [ ]:
my_df = pd.DataFrame(data={
    'coutries': ['usa','india','france','germany'],
    'price': [100, 200, 300, 400]
})
df_data

In [ ]:
i = 0 
for x in df_data['Location'].str.split(', '):
  df_data['nations'][i] = x[-1]
  i += 1
# df_data

In [ ]:
df_data['nations'].replace({'Russia': 'Russian Federation', 'New Mexico': 'USA','Yellow Sea': 'China','Shahrud Missile Test Site': 'Iraq',
 'Pacific Missile Range Facility': 'USA','Barents Sea': 'Russian Federation', 'Gran Canaria': 'USA', 'Zealand': 'new zealand', 'Site':'Iraq', 'North Korea':'Korea, Republic of',  
  'Iran': 'Iraq', 'Pacific Ocean': 'India', 'South Korea':'Korea, Republic of'}, inplace=True)

In [ ]:
i=0
for x in df_data['nations']:
  df_data['country_codes'][i] = countries.get(x).alpha3
  i += 1 
countries.get('Iraq') 
df_data

In [ ]:
df_data['nations'].sample(60)
df_data[df_data['nations']=='Pacific Ocean']

In [ ]:
# i = 0
# for x in df_data['Location'] :
#   y = x.split(',')
#   df_data['nations'][i] = y[-1]
#   i += 1 
# df_data 
for c in countries:
  print(c) 
# countries.get('Korea')

In [ ]:
my_df

In [ ]:
df_data.columns
df_data['Price'] = df_data['Price'].str.replace(',','')
df_data['Price']= df_data['Price'].fillna(0)
df_data['Price'].sample(50)
df_data['Price'] = pd.to_numeric(df_data['Price'])

In [ ]:
grp_data = df_data.groupby('country_codes').sum()

In [ ]:
fig = px.choropleth(grp_data, scope='world', color=grp_data['Price'] ,color_continuous_scale='matter', locationmode='ISO-3', locations= grp_data.index)
# fig. = px.colors.sequential.swatches_continuous()
# plotly.express.colors.diverging color scales as the inputs to
# color_continuous_scale.
fig.show()

# Use a Choropleth Map to Show the Number of Failures by Country


In [ ]:
df_data.info()
df_data3 = df_data[df_data['Mission_Status'] == 'Failure']
counts_df = df_data.groupby('country_codes').count()

In [ ]:
counts_df.rename(columns={'Mission_Status': 'Number of failed launches'}, inplace=True)

In [ ]:
px.choropleth(data_frame=counts_df, locations=counts_df.index, color=counts_df['Number of failed launches'], color_continuous_scale='matter')

In [ ]:
df_data

# Create a Plotly Sunburst Chart of the countries, organisations, and mission status. 

In [ ]:
df_data

In [ ]:
fig = px.sunburst(df_data, path=['countries', 'organisations', 'mission status'], values=['Price'])  

In [ ]:
fig.show()

# Analyse the Total Amount of Money Spent by Organisation on Space Missions

# Analyse the Amount of Money Spent by Organisation per Launch

# Chart the Number of Launches per Year

# Chart the Number of Launches Month-on-Month until the Present

Which month has seen the highest number of launches in all time? Superimpose a rolling average on the month on month time series chart. 

# Launches per Month: Which months are most popular and least popular for launches?

Some months have better weather than others. Which time of year seems to be best for space missions?

# How has the Launch Price varied Over Time? 

Create a line chart that shows the average price of rocket launches over time. 

# Chart the Number of Launches over Time by the Top 10 Organisations. 

How has the dominance of launches changed over time between the different players? 

# Cold War Space Race: USA vs USSR

The cold war lasted from the start of the dataset up until 1991. 

## Create a Plotly Pie Chart comparing the total number of launches of the USSR and the USA

Hint: Remember to include former Soviet Republics like Kazakhstan when analysing the total number of launches. 

## Create a Chart that Shows the Total Number of Launches Year-On-Year by the Two Superpowers

## Chart the Total Number of Mission Failures Year on Year.

## Chart the Percentage of Failures over Time

Did failures go up or down over time? Did the countries get better at minimising risk and improving their chances of success over time? 

# For Every Year Show which Country was in the Lead in terms of Total Number of Launches up to and including including 2020)

Do the results change if we only look at the number of successful launches? 

# Create a Year-on-Year Chart Showing the Organisation Doing the Most Number of Launches

Which organisation was dominant in the 1970s and 1980s? Which organisation was dominant in 2018, 2019 and 2020? 